# Preparação da base de dados para treinar os classificadores

In [ ]:
from src import (
    Folder,
    GetModel,
    SegmentedList,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Rf,
    MLP,
    SVM,
    XGBoost,
    KNN,
    GBM,
    List2Best,
    Save_model
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [ ]:
#adquindo as imagens e seus respectivos rotulos
#dataBasePath = join(DATA,"dados")
dataBasePath = join(DATA,"changed","segmentation")
images, labels = Folder(dataBasePath)

## Caso as imagens não estejam segmentadas, podemos segmentalas

In [ ]:

segmentation_model = GetModel(join("models","best.pt"))
process_images = SegmentedList(images,segmentation_model,is_resized=True) # Por padrão redimensionar garante menor taxa de falha
#process_images = SegmentedList(images,segmentation_model,is_resized=True)

In [ ]:
# com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

def clear_fails(process_images, labels):
    fails = []

    # Iterando de trás para frente
    for i in reversed(range(len(process_images))):
        if process_images[i] is None:
            fails.append(labels[i])
            # Remove os elementos das listas

    return fails

fails = clear_fails(process_images,labels)

print(f"Quantidade: {len(fails)}\nFalhas:",fails)

#msg = "falhas-normais-"
#msg = "falhas-redimensionadas-"

anotation_fails= join("docs","Falhas","seg-2",rf"{msg}-{data}.txt")

with open(anotation_fails, "w") as arquivo:
        for fail in fails:
            arquivo.write(fail + "\n")

# Extraindo as Caracteristicas de interesse da base de dados

In [ ]:
#df = ds2df(process_images,labels)
df = ds2df(valid_images,valid_labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [ ]:

save_results(df,DATA,"resize-969",".csv")

### Carregando nossos dados a partir de um dataset .csv

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"13-03-2024.csv"))

#df.drop("Unnamed: 0",axis=1,inplace=True)
df.columns

# Treinando os classificadores

In [ ]:
rf = Best_rf(df,join("docs","classification"))


### Vamos pegar a melhor versão de cada modelo

In [ ]:
bests_list = []

funcs = [Rf, MLP,SVM,XGBoost,KNN,GBM]
names = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

args = (0.1,0.2,0.3)

for i,func in enumerate(funcs,0):

    for arg in args:
        
        _, accuracy, precision, recall, f1, kappa = func(df,arg)
        bests_list.append((names[i],arg,accuracy, precision, recall, f1, kappa))
    
bests_list

In [ ]:
output = join("docs","classification")

In [ ]:
best = List2Best(bests_list,output,f"{len(valid_images)}.csv",True)
best

In [ ]:
# caso gostemos do resultado e desejemos salvar o modelo
Save_model(model,join(MODEL,'rf'))

# Avaliando o comparativo dos modelos 

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
result_models = File2df(join(output,"not rf.csv"))

#result_models.drop("Unnamed: 0",axis=1,inplace=True)
result_models.columns

In [ ]:
result_models

In [ ]:
rotulos = result_models.Modelo.unique()
rotulos

In [ ]:
# Suponha que 'df' seja o DataFrame contendo os resultados dos modelos

# Rótulos dos modelos
model_labels = ['XGBoost', 'GBM', 'KNN', 'SVM', 'MLP']

# Lista para armazenar os melhores resultados de cada modelo
best_results = []

# Iterar sobre os rótulos dos modelos
for model_label in model_labels:
    # Filtrar o DataFrame para o modelo atual
    model_df = result_models[result_models['Modelo'] == model_label]
    
    # Encontrar a linha com o melhor resultado para o modelo atual
    best_row = model_df.loc[model_df['Total'].idxmax()]
    
    # Adicionar o resultado à lista de melhores resultados
    best_results.append(best_row)

import pandas as pd
# Criar um DataFrame com os melhores resultados
print(best_results)
best_results_df = pd.DataFrame(best_results)

# Exibir o DataFrame com os melhores resultados
print(best_results_df)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Extrair os nomes dos itens e os valores de desempenho
itens = [item[1] for item in result_models]
desempenho = [item[-1] for item in result_models]

# Plotar o gráfico
plt.figure(figsize=(10, 6))
plt.barh(itens, desempenho, color='skyblue')
plt.xlabel('Desempenho')
plt.ylabel('Itens')
plt.title('Comparação de Desempenho entre Itens')
plt.grid(axis='x')
plt.show()